<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Optymalizacja-wybierania-URL/colabs/Optymalizowanie_czasu_scrapowania_URL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Metodyka testowania

Z całego zbior danych o aktorach została wybrana jedynie kolumna z id aktora, a stąd (przy ustalonym dziernie) wybrana została próbka wielkości $2\%$ całego zbioru ($41259$ wierszy).

Test prędkości pobierania linków URL dla id aktorów został przeprowadzony najpierw na bibliotece `BeautifoulSoup`. Następnie dla tej biblioteki zostały sprawdzone różne warianty wybierania linku ze strony. Dla najszybszego z tych wariantów, został przeprowadzone dodatkowe testy używające różnych dostępnych parserów, oraz dodatkowy test osobnej biblioteki `lxml`.

# Wnioski

Z pierwotnie testowanych metod, najszybsza okazała się metoda używająca warunku `if` zamiast `try`, oraz wybierająca ze storny element `src`, a dopiero po jego wybraniu, docinająca uzyskany tekst (zeskalowany link do zdjęcia aktora).

Spośród parserów `html`, `lxml` oraz `html5lib` najszybszy okazał sie parser `lxml`.

Użycie osobnej biblioteki `lxml` zaowocowało jeszcze szybszym pobieraniem zdjęć aktorów. Dla wybranej próbki prędkość ta wynosiła $100$ linków na $1.7$ sekundy. Przeprowadzony został jednorazowy test na próbce wielkości $10\%$ zbioru. Pyspark dla coraz większej próbki, coraz to bardziej paralelizuje wykonywanie kodu, dlatego wydobycie linków do zdjęć wszystkich aktorów ze zbioru powinno zająć nie więcej niż $10$ godzin.

Przy sprawdzaniu dla jakiej części aktorów możemy zdobyć zdjęcia, okazało się, że spośród $41259$ aktorów, dla jedynie $5343$ byliśmy w stanie zdobyć ich zdjęcia, co stanowi $12.95\%$ próbki.

# Uwagi

Testy zostały przeprowadzone w magicznym momencie, gdy czasy wykonywania odpowiednich metod były porównywalne, tj. najszybsza metoda była około $2$ razy szybsza niż najwolniejsza metoda. Jednak ze względu na obciążenie strony, połączenie internetowe oraz możliwie inne czynniki których nie znam i na które nie mamy wpływu. Bardzo często zdarza się że czas wykonywania testowanych metod wacha się (w przypadku najszybszej metody) od 11 minut do ponad godziny. Ponadto bardzo często po kilku godzinach pracy notatnika, zostaje zerwane połączenie z serwerem i praca ustaje.

Biorąc pod uwagę te czynniki, można wnosić że nawet w bardzo optymistycznym scenariuszu gdzie obciążenie serwerów jest bardzo małe i metoda generująca linki URL dla id aktorów działa optymalnie, to nadal czas pracy nad tak dużym zbiorem uniemożliwi pozytywne zakończenie zadania w środowisku colab.

Rozwiązaniem powyższego problemu mogłoby jednak być pracowanie na mniejszych fragmentach całego zbioru. Można np.:
1.   stworzyć plik csv z kolumną z id aktorów, oraz pustą kolumną na linki URL do zdjęć aktorów;
2.   generować linki, oraz co jakiś czas (np. co 1000 aktorów) zapisywać plik csv z wprowadzonymi zmianami na dysku Google;
3.   w przypadku wznowienia pracy na pliku, rozpocząć pracę dla tych wierszy którę pole z linkiem URL mają puste oraz postepować jak w punkcie 2.;
4.   powtarzać punkt 3. aż do momentu gdy linki do zdjęć (lub wartości NULL, w przypadku braku zdjęć) zostaną wygenerowane dla ostatniego aktora.

Równoważnie można podzielić tabelę na mniejsze tabelki i kolejno dla nich generować kolumnę z linkami do zdjęć, a następnie je skleić i dołączyć do zbioru który jest aktualnie w użytku.





# Przygotowanie notatnika

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

Cloning into 'BigData2022-actors'...
remote: Enumerating objects: 1509, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 1509 (delta 141), reused 151 (delta 101), pack-reused 1307
Receiving objects: 100% (1509/1509), 5.98 MiB | 12.67 MiB/s, done.
Resolving deltas: 100% (910/910), done.
mv: cannot move '/content/BigData2022-actors/.' to './.': Device or resource busy
mv: cannot move '/content/BigData2022-actors/..' to './..': Device or resource busy


In [ ]:
!git checkout Optymalizacja-wybierania-URL
# TODO usunąć przy mergu do maina

Branch 'Optymalizacja-wybierania-URL' set up to track remote branch 'Optymalizacja-wybierania-URL' from 'origin'.
Switched to a new branch 'Optymalizacja-wybierania-URL'


## Ładowanie Sparka

In [ ]:
!chmod +x setup_sparka.sh
!./setup_sparka.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=2c5fd61e91173d12358b3d7668f1b4dce6723f888925bcaeb795550159e5508c
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
--2023-01-12 01:29:31--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274099817 

In [ ]:
from lib.pyspark_init import create_spark_context

In [ ]:
spark = create_spark_context()
spark

## Załadowanie danych z pliku o rozszerzeniu .parquet

In [ ]:
from lib.const import JOINED_DATA, DATA_WITH_AWARDS

W poradniku "Tworzenie skrotu do GDrive" w folderze tutorials jest poradnik jak sprawić żeby poniższa lokalizacja się poprawnie ładowała.


In [ ]:
data = spark.read.parquet(JOINED_DATA)
# TODO przepiąć się na nowszy plik gdy będzie on już zapisany na dysku

# Testowanie metod pobierania URL ze strony IMDB

In [ ]:
from lib.url_utils import *

## Wybranie próbki id aktorów

In [ ]:
data = data.select('nconst')

In [ ]:
sample = data.sample(0.02, 2137)

In [ ]:
ids_no = sample.count()
ids_no

41259

In [ ]:
sample.head(5)

[Row(nconst='nm0004822'),
 Row(nconst='nm0007369'),
 Row(nconst='nm0015241'),
 Row(nconst='nm0029357'),
 Row(nconst='nm0031184')]

## Czas pomiędzy metodą z `if`, a metodą z `try`

In [ ]:
%%time
sample_try = sample.withColumn("URL", udf_try_get_link_to_imdb_image("nconst"))
sample_try.head(5)

CPU times: user 8.17 s, sys: 1.15 s, total: 9.31 s
Wall time: 28min 17s


[Row(nconst='nm0004822', URL='https://m.media-amazon.com/images/M/MV5BMjE2ODUwOTIxMl5BMl5BanBnXkFtZTYwMDcyOTU3'),
 Row(nconst='nm0007369', URL=None),
 Row(nconst='nm0015241', URL='https://m.media-amazon.com/images/M/MV5BMGFhNWZhOWEtZjVkNC00ZmIwLTg2NzItZTJhYTViMDIwOTRiXkEyXkFqcGdeQXVyNjg3MTIwODI@'),
 Row(nconst='nm0029357', URL='https://m.media-amazon.com/images/M/MV5BMTk4MTMyMjkwOF5BMl5BanBnXkFtZTgwNDcwNzQ0ODE@'),
 Row(nconst='nm0031184', URL=None)]

In [ ]:
%%time
sample_if = sample.withColumn("URL", udf_if_get_link_to_imdb_image("nconst"))
sample_if.head(5)

CPU times: user 6.66 s, sys: 882 ms, total: 7.54 s
Wall time: 22min 57s


[Row(nconst='nm0004822', URL='https://m.media-amazon.com/images/M/MV5BMjE2ODUwOTIxMl5BMl5BanBnXkFtZTYwMDcyOTU3'),
 Row(nconst='nm0007369', URL=None),
 Row(nconst='nm0015241', URL='https://m.media-amazon.com/images/M/MV5BMGFhNWZhOWEtZjVkNC00ZmIwLTg2NzItZTJhYTViMDIwOTRiXkEyXkFqcGdeQXVyNjg3MTIwODI@'),
 Row(nconst='nm0029357', URL='https://m.media-amazon.com/images/M/MV5BMTk4MTMyMjkwOF5BMl5BanBnXkFtZTgwNDcwNzQ0ODE@'),
 Row(nconst='nm0031184', URL=None)]

## Czas wykonania przy odwoływaniu się do elementu "src"

In [ ]:
%%time
sample_src_try = sample.withColumn("URL", udf_try_get_link_to_imdb_image_src("nconst"))
sample_src_try.head(5)

CPU times: user 7.42 s, sys: 1.02 s, total: 8.44 s
Wall time: 26min 45s


[Row(nconst='nm0004822', URL='https://m.media-amazon.com/images/M/MV5BMjE2ODUwOTIxMl5BMl5BanBnXkFtZTYwMDcyOTU3'),
 Row(nconst='nm0007369', URL=None),
 Row(nconst='nm0015241', URL='https://m.media-amazon.com/images/M/MV5BMGFhNWZhOWEtZjVkNC00ZmIwLTg2NzItZTJhYTViMDIwOTRiXkEyXkFqcGdeQXVyNjg3MTIwODI@'),
 Row(nconst='nm0029357', URL='https://m.media-amazon.com/images/M/MV5BMTk4MTMyMjkwOF5BMl5BanBnXkFtZTgwNDcwNzQ0ODE@'),
 Row(nconst='nm0031184', URL=None)]

In [ ]:
%%time
sample_src_if = sample.withColumn("URL", udf_if_get_link_to_imdb_image_src("nconst"))
sample_src_if.head(5)

CPU times: user 4.9 s, sys: 690 ms, total: 5.59 s
Wall time: 17min 40s


[Row(nconst='nm0004822', URL='https://m.media-amazon.com/images/M/MV5BMjE2ODUwOTIxMl5BMl5BanBnXkFtZTYwMDcyOTU3'),
 Row(nconst='nm0007369', URL=None),
 Row(nconst='nm0015241', URL='https://m.media-amazon.com/images/M/MV5BMGFhNWZhOWEtZjVkNC00ZmIwLTg2NzItZTJhYTViMDIwOTRiXkEyXkFqcGdeQXVyNjg3MTIwODI@'),
 Row(nconst='nm0029357', URL='https://m.media-amazon.com/images/M/MV5BMTk4MTMyMjkwOF5BMl5BanBnXkFtZTgwNDcwNzQ0ODE@'),
 Row(nconst='nm0031184', URL=None)]

Odwołując się do konkretnego elemntu, oraz ucinając końcówkę linku odpowiadającą za rozmiar zdjęcia, uzyskujemy jeszcze lepszy czas wykonywania na naszej próbce.

## Używanie innych parserów

Parser html

In [ ]:
%%time
sample_html = sample.withColumn("URL", udf_if_get_link_to_imdb_image_src("nconst"))
sample_html.head(5)

CPU times: user 7.83 s, sys: 1.05 s, total: 8.89 s
Wall time: 28min 23s


[Row(nconst='nm0004822', URL='https://m.media-amazon.com/images/M/MV5BMjE2ODUwOTIxMl5BMl5BanBnXkFtZTYwMDcyOTU3'),
 Row(nconst='nm0007369', URL=None),
 Row(nconst='nm0015241', URL='https://m.media-amazon.com/images/M/MV5BMGFhNWZhOWEtZjVkNC00ZmIwLTg2NzItZTJhYTViMDIwOTRiXkEyXkFqcGdeQXVyNjg3MTIwODI@'),
 Row(nconst='nm0029357', URL='https://m.media-amazon.com/images/M/MV5BMTk4MTMyMjkwOF5BMl5BanBnXkFtZTgwNDcwNzQ0ODE@'),
 Row(nconst='nm0031184', URL=None)]

Parser lxml

In [ ]:
%%time
sample_lxml = sample.withColumn("URL", udf_if_get_link_to_imdb_image_lxml("nconst"))
sample_lxml.head(5)

CPU times: user 6.86 s, sys: 1 s, total: 7.86 s
Wall time: 25min 28s


[Row(nconst='nm0004822', URL='https://m.media-amazon.com/images/M/MV5BMjE2ODUwOTIxMl5BMl5BanBnXkFtZTYwMDcyOTU3'),
 Row(nconst='nm0007369', URL=None),
 Row(nconst='nm0015241', URL='https://m.media-amazon.com/images/M/MV5BMGFhNWZhOWEtZjVkNC00ZmIwLTg2NzItZTJhYTViMDIwOTRiXkEyXkFqcGdeQXVyNjg3MTIwODI@'),
 Row(nconst='nm0029357', URL='https://m.media-amazon.com/images/M/MV5BMTk4MTMyMjkwOF5BMl5BanBnXkFtZTgwNDcwNzQ0ODE@'),
 Row(nconst='nm0031184', URL=None)]

Parser html5lib

In [ ]:
%%time
sample_html5lib = sample.withColumn("URL", udf_if_get_link_to_imdb_image_html5lib("nconst"))
sample_html5lib.head(5)

CPU times: user 4.7 s, sys: 587 ms, total: 5.28 s
Wall time: 16min 50s


[Row(nconst='nm0004822', URL='https://m.media-amazon.com/images/M/MV5BMjE2ODUwOTIxMl5BMl5BanBnXkFtZTYwMDcyOTU3'),
 Row(nconst='nm0007369', URL=None),
 Row(nconst='nm0015241', URL='https://m.media-amazon.com/images/M/MV5BMGFhNWZhOWEtZjVkNC00ZmIwLTg2NzItZTJhYTViMDIwOTRiXkEyXkFqcGdeQXVyNjg3MTIwODI@'),
 Row(nconst='nm0029357', URL='https://m.media-amazon.com/images/M/MV5BMTk4MTMyMjkwOF5BMl5BanBnXkFtZTgwNDcwNzQ0ODE@'),
 Row(nconst='nm0031184', URL=None)]

## bilbioteka xlml

Test działania metody opartej na osobnej bilbiotece `lxml` której parser był najszybszy dla metod opartych na bibliotece `BeautifulSoup`.

In [ ]:
!pip3 install cssselect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%time
sample_lib_lxml = sample.withColumn("URL", udf_lxml_get_link_to_image("nconst"))
sample_lib_lxml.head(5)

CPU times: user 10.1 s, sys: 1.4 s, total: 11.4 s
Wall time: 32min 16s


[Row(nconst='nm0004822', URL='https://m.media-amazon.com/images/M/MV5BMjE2ODUwOTIxMl5BMl5BanBnXkFtZTYwMDcyOTU3'),
 Row(nconst='nm0007369', URL=None),
 Row(nconst='nm0015241', URL='https://m.media-amazon.com/images/M/MV5BMGFhNWZhOWEtZjVkNC00ZmIwLTg2NzItZTJhYTViMDIwOTRiXkEyXkFqcGdeQXVyNjg3MTIwODI@'),
 Row(nconst='nm0029357', URL='https://m.media-amazon.com/images/M/MV5BMTk4MTMyMjkwOF5BMl5BanBnXkFtZTgwNDcwNzQ0ODE@'),
 Row(nconst='nm0031184', URL=None)]

Metoda ta okazuje się być najszybsza spośród testowanych.

## Ilość zdjęć

Sprawdzamy dla jak wielu spośród wszystkich aktorów z naszej próbki jesteśmy w stanie zdobyć link do zdjęcia z IMDB.

In [ ]:
from pyspark.sql.functions import col, when, count

In [ ]:
data_no_nulls = sample_lib_lxml.filter(col("URL").isNotNull())

In [ ]:
%%time
links_no = data_no_nulls.count()

CPU times: user 55.9 s, sys: 7.7 s, total: 1min 3s
Wall time: 3h 11min 9s


In [ ]:
links_no

5343

In [ ]:
100 * links_no/ids_no

12.949901839598633